In [1]:
import AFQ.data as afqd
import cloudknot as ck
import importlib
import s3fs

/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/dipy/stats/__init__.py:7: UserWarning: The `dipy.stats` module is still under heavy development and functionality, as well as the API is likely to change in future versions of the software
  warnings.warn(w_string)
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials f

In [2]:
importlib.reload(ck)

<module 'cloudknot' from '/Users/richford/projects/cloudknot/cloudknot/cloudknot/__init__.py'>

In [3]:
hbn = afqd.HBNSite(site="Site-RU", subjects="all")

/Users/richford/miniconda3/envs/cloudknot_qsiprep/lib/python3.7/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


Retrieving subject S3 keys
[########################################] | 100% Completed |  5min 19.9s


In [24]:
ck.get_region()

'us-west-2'

In [25]:
ck.set_region('us-east-1')

In [26]:
fs = s3fs.S3FileSystem()
fs.touch("hbn-derivatives/qsiprep-ck/useless_file")

## Define the preprocessing function

In [27]:
def preprocess_hbn_ru_subject(subject_id):
    import AFQ.data as afqd
    import os
    import subprocess
    import sys
    from bids import BIDSLayout
    from s3fs import S3FileSystem
        
    hbn_ru = afqd.HBNSite(site="Site-RU", subjects=subject_id)
    hbn_ru.download("./hbn-ru")
    fs = S3FileSystem()
    
    layout = BIDSLayout("./hbn-ru")
    dwi_subs = layout.get(return_type="id", target="subject", datatype="dwi")
    dwi_subs = ["sub-" + sub for sub in dwi_subs]
    
    # HBN has other files that we don't need for dMRI preproc and whose presence will break qsiprep
    # Get rid of them
    s0 = hbn_ru.subjects[0]
    tracew_dwi_files = [file for key, file in s0.files["raw"].items() if "TRACEW" in key]
    fmri_files = [file for key, file in s0.files["raw"].items() if "/func/" in key]
    fmri_files += [file for key, file in s0.files["raw"].items() if "/fmap/" in key and "fMRI" in key]
    
    for fname in fmri_files + tracew_dwi_files:
        os.remove(fname)
    
    if subject_id in dwi_subs:
        response = subprocess.run(
            [
                "qsiprep",
                "--output-resolution",
                "1.8",
                "--participant-label",
                subject_id,
                "-w",
                "./hbn-wrk",
                "--nthreads",
                "8",
                "--omp-nthreads",
                "8",
                "--force-spatial-normalization",
                "--dwi-denoise-window",
                "5",
                "--combine-all-dwis",
                "--unringing-method",
                "mrdegibbs",
                "./hbn-ru",
                "./hbn-preproc",
                "participant",
            ],
            check=True
        )
        
        output_dir = "hbn-derivatives/qsiprep-ck"            
        fs.put(f"./hbn-preproc/qsiprep/{subject_id}",
               "/".join([output_dir, subject_id]), recursive=True)
        fs.put(f"./hbn-preproc/qsiprep/{subject_id}.html",
               "/".join([output_dir, subject_id + ".html"]))

        return {subject_id: True}
    else:
        return {subject_id: False}

# Create a cloudknot DockerImage instance

where the base_image is a customized version of qsiprep
This version of the qsiprep docker image is located on AWS ECR with image URI:
[454929164628.dkr.ecr.us-east-1.amazonaws.com/qsiprep:nrdg](https://console.aws.amazon.com/ecr/repositories/qsiprep/?region=us-east-1)

In [28]:
di = ck.DockerImage(
    name="preprocess-hbn-ru-subject",
    func=preprocess_hbn_ru_subject,
    base_image="qsiprep:nrdg",
    github_installs=["https://github.com/richford/pyAFQ.git@s3-bids-fetch",
                     "https://github.com/bids-standard/pybids.git@0.9.3"],
    overwrite=True,
)

## Build, tag, and push the Docker image

In [29]:
di.build(tags=["hbn-ru-preproc-20200723"])

In [30]:
repo = ck.aws.DockerRepo(name=ck.get_ecr_repo())

In [31]:
repo.repo_uri

'454929164628.dkr.ecr.us-east-1.amazonaws.com/cloudknot'

In [32]:
# The very first time you run this, this command could take
# a few hours because the docker image is large
di.push(repo=repo)

In [33]:
len(hbn.subjects)

1227

## Create the Knots

In [40]:
sub_ids = [s.subject_id for s in hbn.subjects]

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
chunk_knots = {
    n: {
        # Specify bid_percentage to use Spot instances
        # And make sure the volume size is large enough. 50-55 GB seems about right for HBN preprocessing. YMMV.
        "knot": ck.Knot(
            name=f"hbn_preproc_20200804_ch{n}",
            docker_image=di,
            pars_policies=('AmazonS3FullAccess',),
            bid_percentage=100,
            memory=64000,
            job_def_vcpus=8,
            volume_size=55
        ),
        "subjects": chunk
    } for n, chunk in enumerate(chunks(sub_ids, 320))
}

## Get subject IDs and predict output

We'll pass these subject IDs to `knot.map()`

In [ ]:
sub_ids = [s.subject_id for s in hbn.subjects]
sub_ids

Some HBN subjects do not have dwi data. These will fail and return `{subject_id: False}`. Let's predict how many will fail.

In [ ]:
dwi_predict = {
    sub.subject_id: any(["dwi" in s3key for s3key in sub.s3_keys["raw"]])
    for sub in hbn.subjects
}
dwi_predict

## Submit the jobs and check on results

In [41]:
for i in chunk_knots.keys():
    chunk_knots[i] = {
        "knot": chunk_knots[i]["knot"],
        "subjects": chunk_knots[i]["subjects"],
        "results": chunk_knots[i]["knot"].map(chunk_knots[i]["subjects"])
    }

In [ ]:
ck.set_region('us-east-1')

In [ ]:
jobs = knot.jobs

In [ ]:
j0 = jobs[0]

In [ ]:
j0.terminate("Output to S3 is undesirable.")

In [ ]:
knot.view_jobs()

In [ ]:
print([j.status for j in knot.jobs])

## Results

The results are dicts where the keys are the subject IDs and the values report sucess or failure

In [ ]:
result_futures.result()

## Figure out how many subjects we have in the entire study to support some cost estimates

In [ ]:
all_sites = {}

for site in ["Site-SI", "Site-CBIC", "Site-RU", "Site-CUNY"]:
    all_sites[site] = afqd.HBNSite(site=site)
    print(f"{site}: {len(all_sites[site]._all_subjects)}")

## When you're done, clobber the knot

In [ ]:
knot.clobber(clobber_pars=True)